In [10]:
import pickle
import os
from datetime import datetime
import pandas as pd
import numpy as np

In [11]:
model_path = r"C:\Users\ramna\Desktop\Bike Demand Analyzer\models\xgboost_regressor_r2_0_94_v3.pkl"
model = pickle.load(open(model_path, "rb"))

In [12]:
data = [
    [-0.17373643,  0.73564541, -0.41047115,  0.92170813, -0.67552504,
       -0.18781326, -0.19904527, -0.20980267, -0.21450357,  4.80270071,
       -0.20501252, -0.20217449, -0.20940699, -0.21294615, -0.20901068,
       -0.21019774, -0.20661954, -0.20940699, -0.20781798, -0.20821618,
       -0.20940699, -0.21098603, -0.21177188, -0.21059219, -0.20053759,
       -0.21333638, -0.20135745, -0.21098603, -0.20901068, -0.20501252,
        1.72191666, -0.57582707, -0.57418668,  0.23069934,  0.17946416,
       -0.28793668, -0.30345966, -0.30136113,  3.28885189, -0.296831  ,
       -0.30345966, -0.30226169, -0.29652734, -0.30465451, -0.30495274,
       -0.30465451,  1.58892357],
    [ 0.74422235,  0.98131174, -0.71451814,  0.92170813, -0.67552504,
       -0.18781326, -0.19904527, -0.20980267,  4.66192707, -0.20821618,
       -0.20501252, -0.20217449, -0.20940699, -0.21294615, -0.20901068,
       -0.21019774, -0.20661954, -0.20940699, -0.20781798, -0.20821618,
       -0.20940699, -0.21098603, -0.21177188, -0.21059219, -0.20053759,
       -0.21333638, -0.20135745, -0.21098603, -0.20901068, -0.20501252,
       -0.5807482 , -0.57582707, -0.57418668,  0.23069934,  0.17946416,
       -0.28793668, -0.30345966, -0.30136113, -0.30405747, -0.296831  ,
       -0.30345966, -0.30226169,  3.37237029, -0.30465451, -0.30495274,
       -0.30465451,  1.58892357],
    [-0.7662371 ,  0.34257927,  2.32595175, -0.6218116 , -0.63866374,
       -0.18781326,  1.1547098 , -0.20980267, -0.21450357, -0.20821618,
       -0.20501252, -0.20217449, -0.20940699, -0.21294615, -0.20901068,
       -0.21019774, -0.20661954, -0.20940699, -0.20781798, -0.20821618,
       -0.20940699, -0.21098603, -0.21177188,  4.7485142 , -0.20053759,
       -0.21333638, -0.20135745, -0.21098603, -0.20901068, -0.20501252,
       -0.5807482 , -0.57582707,  1.74159386,  0.23069934,  0.17946416,
       -0.28793668, -0.30345966, -0.30136113, -0.30405747, -0.296831  ,
       -0.30345966, -0.30226169, -0.29652734, -0.30465451, -0.30495274,
        3.28240667,  1.58892357]
]

prediction = model.predict(data)
list(prediction)

[np.float32(15.92996), np.float32(25.381313), np.float32(14.503452)]

# Load Scaler

In [13]:
sc_dump_path = r"C:\Users\ramna\Desktop\Bike Demand Analyzer\models\sc.pkl"

scaler = pickle.load(open(sc_dump_path, "rb"))

In [14]:
scaler

StandardScaler()

## Load Column

In [15]:
column_path = r"C:\Users\ramna\Desktop\Bike Demand Analyzer\models\columns.pkl"
columns = pickle.load(open(column_path, "rb"))
columns

['Temperature',
 'Humidity',
 'Wind_speed',
 'Visibility',
 'Solar_Radiation',
 'Rainfall',
 'Snowfall',
 'Hour_1',
 'Hour_2',
 'Hour_3',
 'Hour_4',
 'Hour_5',
 'Hour_6',
 'Hour_7',
 'Hour_8',
 'Hour_9',
 'Hour_10',
 'Hour_11',
 'Hour_12',
 'Hour_13',
 'Hour_14',
 'Hour_15',
 'Hour_16',
 'Hour_17',
 'Hour_18',
 'Hour_19',
 'Hour_20',
 'Hour_21',
 'Hour_22',
 'Hour_23',
 'Seasons_Spring',
 'Seasons_Summer',
 'Seasons_Winter',
 'Holiday_No Holiday',
 'Functioning_Day_Yes',
 'month_2',
 'month_3',
 'month_4',
 'month_5',
 'month_6',
 'month_7',
 'month_8',
 'month_9',
 'month_10',
 'month_11',
 'month_12',
 'weekdays_weekend_1']

## User Input

## Converting User Data into Model Consumable Format

In [16]:
def preprocess_input(input_data):
    # Create a DataFrame from the input data
    input_df = pd.DataFrame([input_data])

    # Convert "Date" to "year", "month", and "day"
    input_df['Date'] = pd.to_datetime(input_df['Date'], format="%d-%m-%Y")
    input_df['year'] = input_df['Date'].dt.year
    input_df['month'] = input_df['Date'].dt.month
    input_df['day'] = input_df['Date'].dt.day_name()  # Day name for weekdays_weekend

    # Create the weekdays_weekend column
    input_df['weekdays_weekend'] = input_df['day'].apply(lambda x: 1 if x in ['Saturday', 'Sunday'] else 0)

    # Drop the 'Date', 'day', and 'year' columns as they are not needed for prediction
    input_df = input_df.drop(['Date', 'day', 'year'], axis=1)

    # One-hot encode the categorical features
    categorical_features = ['Hour', 'Seasons', 'Holiday', 'Functioning_Day', 'month', 'weekdays_weekend']
    for col in categorical_features:
        dummies = pd.get_dummies(input_df[col], prefix=col, drop_first=True)
        input_df = pd.concat([input_df, dummies], axis=1)
        input_df = input_df.drop([col], axis=1)

    # Ensure all expected columns are present
    for col in columns:
        if col not in input_df.columns:
            input_df[col] = 0

    return input_df

# Example input_data
input_data = {
    'Hour': 2,
    'Temperature': -6.0	,
    'Humidity': 39,
    'Wind_speed': 1.0,
    'Visibility': 2000,
    'Solar_Radiation': 0.0,
    'Rainfall': 0.0,
    'Snowfall': 0.0,
    'Seasons': 'Winter',
    'Holiday': 'No Holiday',
    'Functioning_Day': 'Yes',
    'Date': '01-12-2017'
}



In [17]:
def make_prediction(input_data):
    # Preprocess the input
    processed_input = preprocess_input(input_data)

    # Scale the input features
    scaled_input = scaler.transform(processed_input)

    # Make the prediction (predicting the square root of the bike count in scaled form)
    predicted_scaled_sqrt_value = model.predict(scaled_input).reshape(-1, 1)

    # Inverse transform the prediction to get the original scale
    predicted_sqrt_value = scaler.inverse_transform(np.concatenate([predicted_scaled_sqrt_value, scaled_input[:, 1:]], axis=1))[:, 0].reshape(-1, 1)

    # Square the prediction to get the original bike count
    predicted_value = (predicted_sqrt_value ** 2).tolist()

    # Return the predicted value
    return round(predicted_value[0][0])


In [18]:
# Make the prediction
predicted_value = make_prediction(input_data)
print(f"Predicted Rented Bike Count: {predicted_value}")

Predicted Rented Bike Count: 5547
